# Multi-Objective Capacitated VRP (MOVRP) Problem Using Multi-Objective Evolutionary Algorithms (MOEAs)

### Global Functions

In [1]:
import math
import numpy as np

def parse_vrp_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Strip whitespace and filter empty lines
    lines = [line.strip() for line in lines if line.strip()]

    # Parse metadata
    metadata = {}
    i = 0
    while not lines[i].startswith('NODE_COORD_SECTION'):
        if ':' in lines[i]:
            key, value = lines[i].split(':', 1)
            metadata[key.strip()] = value.strip().strip('"')
        i += 1

    # Parse node coordinates (1-based in file, but we'll make 0-based list)
    coords = []
    i += 1  # Skip section header
    while not lines[i].startswith('DEMAND_SECTION'):
        parts = lines[i].split()
        if len(parts) == 3:
            node_id = int(parts[0]) - 1  # Make 0-based
            x, y = int(parts[1]), int(parts[2])
            # Insert at correct index in case not sequential (though they usually are)
            while len(coords) <= node_id:
                coords.append(None)
            coords[node_id] = (x, y)
        i += 1

    # Parse demands (0-based list)
    demands = []
    i += 1  # Skip section header
    while not lines[i].startswith('DEPOT_SECTION'):
        parts = lines[i].split()
        if len(parts) == 2:
            node_id = int(parts[0]) - 1  # Make 0-based
            demand = int(parts[1])
            while len(demands) <= node_id:
                demands.append(None)
            demands[node_id] = demand
        i += 1

    # Parse depot(s) — usually just one, 1-based
    depots = []
    i += 1  # Skip section header
    while i < len(lines) and lines[i] != 'EOF':
        depot = int(lines[i])
        if depot == -1:
            break
        depots.append(depot - 1)  # Make 0-based
        i += 1

    # Compute distance matrix (Euclidean 2D)
    dimension = int(metadata.get('DIMENSION', len(coords)))
    dist_matrix = np.zeros((dimension, dimension))
    for a in range(dimension):
        for b in range(dimension):
            x1, y1 = coords[a]
            x2, y2 = coords[b]
            dist_matrix[a][b] = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

    return {
        'metadata': metadata,
        'coords': coords,  # List of (x, y) tuples, 0-based (index 0 = node 1)
        'demands': demands,  # List of integers, 0-based
        'depots': depots,  # List of 0-based indices (usually [0] for depot at node 1)
        'dist_matrix': dist_matrix  # Numpy array of floats
    }

# Example usage for your files
small_data = parse_vrp_file('data/small.vrp')
medium_data = parse_vrp_file('data/medium.vrp')
large_data = parse_vrp_file('data/large.vrp')

# Print some extracted info from large (based on the provided sample)
print(large_data['metadata'])
print("Depot index:", large_data['depots'][0])
print("Sample coords (first 5):", large_data['coords'][:5])
print("Sample demands (first 5):", large_data['demands'][:5])
print("Sample distance (node 0 to 1):", large_data['dist_matrix'][0][1])

{'NAME': 'X-n101-k25', 'COMMENT': 'Generated by Uchoa, Pecin, Pessoa, Poggi, Subramanian, and Vidal (2013)', 'TYPE': 'CVRP', 'DIMENSION': '101', 'EDGE_WEIGHT_TYPE': 'EUC_2D', 'CAPACITY': '206'}
Depot index: 0
Sample coords (first 5): [(365, 689), (146, 180), (792, 5), (658, 510), (461, 270)]
Sample demands (first 5): [0, 38, 51, 73, 70]
Sample distance (node 0 to 1): 554.1137067425782


## NSGA-II Implementation

In [ ]:
import random
import copy


class NsgaII:
    """
    NSGA-II algorithm for Multi-Objective Capacitated VRP (MOVRP).
    Each individual is represented as a dictionary containing:
    - 'customer_sequence': permutation of customer indices (excluding depot)
    - 'routes': list of routes (each route is a list of node indices)
    - 'objectives': [total_distance, max_route_length]
    - 'rank': non-domination rank
    - 'crowding_distance': crowding distance value
    """

    def __init__(self, data, pop_size=100, generations=100, crossover_rate=0.7, mutation_rate=0.2):
        self.data = data
        self.pop_size = pop_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.population = []

    def split_routes(self, customer_sequence):
        """
        Split the customer sequence into feasible routes based on vehicle capacity.
        :param customer_sequence: List of customer indices (excluding depot)
        :return: List of routes (each route is a list of node indices)
        """
        capacity = int(self.datea['metadata']['CAPACITY'])
        demands = self.data['demands']
        routes = []
        route = [0]
        load = 0
        for customer in customer_sequence:
            demand = demands[customer] # Demand of current customer
            if load + demand > capacity: # If adding this customer exceeds capacity
                route.append(0) # Return to depot
                routes.append(route) # Save current route
                route = [0, customer] # Start new route
                load = demand # Reset load to current customer's demand
            else:
                route.append(customer) # Add customer to current route
                load += demand # Update load
        route.append(0) # Return to depot at end of last route
        routes.append(route) # Save last route
        return routes
    
    def total_distance(self,routes):
        """
        Calculate the total distance of all routes.
        :param routes: List of routes 
        : return: Total distance(float)
        """
        dist_matrix = self.data['dist_matrix']
        return sum(
            sum(dist_matrix[route[i]][route[i+1]] for i in range(len(route)-1))
            for route in routes
        )
    
    def max_route_length(self,routes):
        """ 
        Calculate the length of the longest route.
        :param routes: List of routes
        : return: Maximum route length(float)
        """
        dist_matrix = self.data[dist_matrix]
        return max(
            sum(dist_matrix[route[i]][route[i+1]] for i in range(len(route)-1))
            for route in routes
        )
    
    def generate_initial_population(self):
        """
        Generate an initial population of random VRP individuals.
        :return: List of individuals (dicts)
        """
        n_customers = len(self.data['coords']) - 1  # Number of customers (excluding depot)
        population = []  # List to store individuals
        for _ in range(self.pop_size):  # For each individual
            perm = list(range(1, n_customers + 1))  # Customer indices (1-based)
            random.shuffle(perm)  # Shuffle to create random permutation
            individual = {
                'customer_sequence': perm,  # Permutation of customers
                'routes': self.split_routes(perm),  # Split into feasible routes
                'objectives': None,  # Objective values (to be set later)
                'rank': None,  # Non-domination rank
                'crowding_distance': None  # Crowding distance
            }
            population.append(individual)  # Add individual to population
        return population
    
    def evaluate_objectives(self, population):
        """ 
        Evaluate objectives for each individual in the population.
        :param population: List of individuals 
        """
        for individual in population:  # For each individual
            total_dist = self.total_distance(individual['routes'])  # Calculate total distance
            max_length = self.max_route_length(individual['routes'])  # Calculate max route length
            individual['objectives'] = [total_dist, max_length]  # Store objectives

    def dominates(self, ind_a, ind_b):
        """
        Check if individual ind_a dominates individual ind_b.
        :param ind_a: Individual (dict)
        :param ind_b: Individual (dict)
        :return: True if ind_a dominates ind_b, else False
        """
        return all(a <= b for a, b in zip(ind_a['objectives'], ind_b['objectives'])) and \
               any(a < b for a, b in zip(ind_a['objectives'], ind_b['objectives']))
    
    def non_dominated_sort(self,population):
        """
        Assign non-dominate ranks to the population.
        :param population: List f individuals 
        """
        for individual in population:
            individual['domination_count'] = 0 # Number of individuals that dominate this one
            individual['dominated_solutions'] = [] # List of individuals this one dominates
            individual['rank'] = None  # Non-domination rank
            for other in population: # Compare with every other individual
                if self.dominates(individual, other): # If individual dominates other
                    individual['dominated_solutions'].append(other) # Add to dominated list
                elif self.dominates(other, individual): # If other dominates individual
                    individual['domination_count'] += 1 # Increment domination count
            if individual['domination_count'] == 0: # If no one dominates this individual
                individual['rank'] = 1 # Assign rank 1
        front = 1 # Start with first front 
        while True:
            current_front = [ind for ind in population if ind['rank'] == front] # Individual in current front
            if not current_front: # If no individuals in this front, we're done
                break 
            for ind in current_front: # For each individual in current front 
                for dominated in ind['dominated_solutions']: # For each individual it dominates 
                    dominated['domination_count'] -= 1 # Decrement their domination count
                    if dominated['domination_count'] == 0: # If they are no longer dominated 
                        dominated['rank'] = front + 1 # Assign next front rank
            front += 1 # Move to next front

    def calculate_crowding_distance(self, population):
        """
        Calculate crowding distance for each individual in the population.
        :param population: List of individuals 
        """
        for front in set(ind['rank'] for ind in population if ind['rank'] is not None):  # For each front
            front_inds = [ind for ind in population if ind['rank'] == front]  # Individuals in this front
            if len(front_inds) <= 2: # If only two or fewer individuals 
                for ind in front_inds:
                    ind['crowding_distance'] = float('inf')  # Assign infinite distance
                continue
            num_objectives = len(front_inds[0]['objectives'])  # Number of objectives
            for i in range(num_objectives):  # For each objective
                front_inds.sort(key=lambda x: x['objectives'][i])  # Sort by this objective
                front_inds[0]['crowding_distance'] = front_inds[-1]['crowding_distance'] = float('inf')  # Boundary points
                for j in range(1,len(front_inds)-1): # For each non-boundary individual 
                    if front_inds[-1]['objectives'][i] == front_inds[0]['objectives'][i]:
                        continue # Skip if all values are the same
                    distance = (front_inds[j+1]['objectives'][i] - front_inds[j-1]['objectives'][i]) / \
                               (front_inds[-1]['objectives'][i] - front_inds[0]['objectives'][i])  # Normalized distance
                    if front_inds[j]['crowding_distance'] is None:
                        front_inds[j]['crowding_distance'] = 0  # Initialize if needed
                    front_inds[j]['crowding_distance'] += distance  # Add to crowding distance

    


    



## SPEA2 Implementation

In [3]:
## TODO ADD ALGO